# LinkedIn Hiring Rate Analysis: MENAAP Labor Market Dynamics (2018-2024)
The analysis of LinkedIn Hiring Rate (LHR) data reveals distinct patterns across Middle East & North Africa, Afghanistan & Pakistan (MENAAP) economies, with notable variations in labor market recovery. The countries analysed are Algeria, Bahrain, Egypt, Jordan, Morocco, Qatar, Saudi Arabia, Tunisia, United Arab Emirates

In [10]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import ColumnDataSource, HoverTool, Tabs, TabPanel, Toggle, CustomJS
from bokeh.palettes import Category20
from bokeh.layouts import column
import openpyxl

In [2]:
def clean_linkedin_data(excel_file):
    """Clean and prepare LinkedIn data from Excel file."""
    try:
        data_corrected = pd.read_excel(excel_file, sheet_name="2A - LHR by Ctry", header=3)

        data_cleaned = data_corrected.rename(columns={
            'Unnamed: 1': 'Month',
            'Unnamed: 2': 'Country',
            'Unnamed: 3': 'LHR (YOY)'
        })

        data_cleaned = data_cleaned.dropna(subset=['Month', 'Country', 'LHR (YOY)'])
        data_cleaned = data_cleaned.iloc[1:]
        data_cleaned = data_cleaned.drop(columns=['Unnamed: 0'])

        # Clean up whitespace and standardize country names
        data_cleaned["Country"] = data_cleaned["Country"].str.strip()
        data_cleaned["Country"] = data_cleaned["Country"].replace({
            "Turkey": "Turkiye",
            "Türkiye": "Turkiye"
        })

        return data_cleaned

    except Exception as e:
        print(f"Error during data cleaning: {str(e)}")
        return None



In [15]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import (
    ColumnDataSource, HoverTool, Tabs, TabPanel, Toggle, CustomJS, Button
)
from bokeh.layouts import column
from bokeh.palettes import Category20
import pandas as pd

def create_country_plots(excel_file):
    """Create interactive plots showing LinkedIn Hiring Rate by Country with 2022+ filter toggle and download button."""
    output_notebook()
    
    country_map = {
        'DZ': 'Algeria', 
        'BH': 'Bahrain', 
        'EG' : 'Egypt',
        'IQ': 'Iraq',
        'JO': 'Jordan',
        'MA': 'Morocco',
        'QA': 'Qatar',
        'SA': 'Saudi Arabia',
        'TN': 'Tunisia',
        'AE': 'United Arab Emirates',
        'PK': 'Pakistan',
        'US': 'United States'
    }
    
    try:
        df = clean_linkedin_data(excel_file)
        
        if df is None:
            print("Data cleaning failed. Please check your Excel file.")
            return
        
        # Compute global y-axis range for all countries
        global_min = df['LHR (YOY)'].min()
        global_max = df['LHR (YOY)'].max()

        tabs = []

        for idx, (country_code, country_name) in enumerate(country_map.items()):
            country_data = df[df['Country'] == country_name].copy()
            if country_data.empty:
                print(f"No data available for {country_name}.")
                continue

            # Keep datetime format for plotting
            country_data['Month'] = pd.to_datetime(country_data['Month'])
            country_data = country_data.sort_values('Month')

            # Add string version of Month for download
            country_data['Month_str'] = country_data['Month'].dt.strftime('%Y-%m')

            # Prepare sources
            full_data = country_data
            filtered_data = full_data[full_data['Month'] >= '2022-01-01']

            source = ColumnDataSource(full_data)
            source_filtered = ColumnDataSource(filtered_data)
            # Download source: string Month only
            source_download = ColumnDataSource(
                full_data[['Country', 'Month_str', 'LHR (YOY)']].rename(columns={'Month_str': 'Month'})
            )

            # Create plot
            p = figure(
                title=f"LinkedIn Hiring Rate in {country_name}",
                x_axis_type='datetime',
                width=800,
                height=500,
                background_fill_color="#f8f9fa",
                y_range=(global_min, global_max)
            )

            p.line(
                x='Month',
                y='LHR (YOY)',
                source=source,
                line_width=3,
                color=Category20[20][idx % 20]
            )

            hover = HoverTool(tooltips=[
                ("Month", "@Month{%b %Y}"),
                ("LHR (YOY)", "@{LHR (YOY)}{0.00}%")
            ], formatters={"@Month": "datetime"}, mode='vline')
            p.add_tools(hover)

            p.xaxis.axis_label = 'Month'
            p.yaxis.axis_label = 'LinkedIn Hiring Rate (YOY %)'
            p.xaxis.axis_label_text_font_size = '12pt'
            p.yaxis.axis_label_text_font_size = '12pt'
            p.xaxis.major_label_text_font_size = '10pt'
            p.yaxis.major_label_text_font_size = '10pt'
            p.title.text_font_size = '14pt'
            p.grid.grid_line_color = "gray"
            p.grid.grid_line_alpha = 0.3

            # Toggle button
            toggle = Toggle(label="Show only from 2022", button_type="success", active=False)

            callback = CustomJS(args=dict(
                toggle=toggle,
                source=source,
                full=source.data,
                filtered=source_filtered.data
            ), code="""
                source.data = toggle.active ? filtered : full;
                source.change.emit();
                toggle.label = toggle.active ? "Show full range" : "Show only from 2022";
                toggle.button_type = toggle.active ? "warning" : "success";
            """)
            toggle.js_on_change("active", callback)

            # Download button
            download_button = Button(label="Download CSV", button_type="primary")

            download_js = CustomJS(args=dict(source=source_download, name=country_name), code="""
                const data = source.data;
                const cols = ["Country", "Month", "LHR (YOY)"];
                const nrows = data[cols[0]].length;
                let csv = cols.join(",") + "\\n";
                for (let i = 0; i < nrows; i++) {
                    let row = cols.map(col => data[col][i]);
                    csv += row.join(",") + "\\n";
                }
                const blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
                const link = document.createElement("a");
                link.href = URL.createObjectURL(blob);
                const filename = "linkedin_HR_" + name.replace(/ /g, "_") + ".csv";
                link.download = filename;
                link.click();
            """)
            download_button.js_on_click(download_js)

            # Combine all widgets
            layout = column(toggle, download_button, p)
            tab = TabPanel(child=layout, title=country_name)
            tabs.append(tab)

        if tabs:
            show(Tabs(tabs=tabs))
        else:
            print("No data available for any of the specified countries.")
    
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print("Please check your Excel file structure and column names.")




In [16]:
from bokeh.io import output_notebook, show
output_notebook()
file_path = '../data/LinkedIn/LinkedIn_LHR by Industry_Feb2025.xlsx'

create_country_plots(file_path)

Loading BokehJS ...

Loading BokehJS ...

### Insights

#### General Note

To interpret these results appropriately, it's important to consider LinkedIn’s guidance on the level of disaggregation at which the hiring rate estimates are valid:

* For **Algeria, Bahrain, Egypt, Jordan, Morocco, Qatar, Saudi Arabia, and Tunisia**, LinkedIn deems the hiring rate valid across **tech-related industries**.
* For the **United Arab Emirates**, estimates are considered valid across **a broader set of industries**.
* For **Iraq and Pakistan**, LinkedIn only considers the estimates valid when presented at the **national level**.

The following insights are presented at the **national level** and are therefore valid for all countries.


#### COVID-19 Impact (2020)

* All markets experienced a **significant contraction** in 2020.
* Uniform negative growth rates were observed across countries (\~**-0.65%**).
* **Morocco (-0.8045%)** and **Tunisia (-0.7936%)** recorded the largest declines.
* Recovery patterns began to diverge in magnitude, though they followed similar overall trends.

#### Post-COVID Recovery Surge (2021)

* A **sharp increase in April 2021** hiring rates is visible across countries, driven by **year-over-year (YoY) calculations** ([see methodology](https://economicgraph.linkedin.com/content/dam/me/economicgraph/en-us/PDF/linkedin-hiring-rate-methodology.pdf)).

  * This spike is primarily due to **base effects** from April 2020, when hiring plummeted.
  * The surge reflects a **rebound** in labor market activity, not necessarily a hiring boom.

* Peak YoY growth rates in mid-2021:

  * **Morocco:** 3.503% (April)
  * **Algeria:** 3.4545% (April)
  * **Tunisia:** 3.118% (April)
  * **Pakistan:** 2.48% (May)
  * **Iraq:** 2.1892% (April)
  * **UAE:** 2.0987% (April)
  * **Qatar:** 2.0857% (April)
  * **Jordan:** 2.0499% (April)
  * **Saudi Arabia:** 1.9015% (April)
  * **Egypt:** 1.4289% (April)
  * **Bahrain:** 1.43% (May)

* Most countries peaked in **April**, except **Pakistan** and **Bahrain**, which peaked in **May**.

* The synchronized peaks suggest a **regional “catch-up” effect** after COVID-19 disruptions.

#### Market Stabilization Patterns (2022–2024)

* **Morocco**: Continued growth through 2022 until **December**, followed by mostly negative or near-zero growth.
* **Algeria**: Stable trend around zero; **slightly negative** since **December 2023**.
* **Tunisia**: Positive growth in 2022 with a **May peak (0.91%)**; mostly negative growth from **April 2023** onward.
* **Pakistan**: Positive growth throughout 2022 until **October 2023**, then fluctuating around zero with short periods of positive/negative growth.
* **Iraq**: Mild positive growth during 2022; slight negative or near-zero growth through 2023–2024 with a **July 2024 peak (0.12%)**, then stable around zero.
* **United Arab Emirates**: Positive growth during the **first three quarters of 2022**, followed by **slightly negative** values near zero.
* **Qatar**: Dynamic trend with early 2022 growth, a **dip in December 2022 (-0.04%)**, followed by a **peak in December 2023 (+0.47%)**, and mostly negative but near-zero values afterward.
* **Jordan**: Positive growth during 2022; **persistent negative values from January 2023 to September 2024** (except **May 2023**), followed by slight improvement post-October 2024.
* **Saudi Arabia**: Fluctuating around zero since 2022; notable variations include: **May 2022:** +0.45%, **April 2023:** -0.27%, **June 2023:** -0.26%, **July 2023:** +0.36%.
* **Egypt**: Some growth in early 2022; mostly negative from late 2022 to **April 2024**. Beginning **July 2024**, stable near-zero values with slight improvements.
* **Bahrain**: Some growth in 2022, but **almost all periods since January 2023 show negative growth**.


